# Про градиентный спуск

Сравнить сложность точного решения задачи линейной регрессии для случая квадратичной ошибки и сложность решения этой же задачи методом градиентного спуска (при условии, что градиентный спуск сходится с K шагов, в задаче есть M признаков и длина обучающей выборки равна N. N>K и N>M)

# Про FWL-Теорему

Кажется, я успешно успел вам наврать на лекции про "отпиливание" зависимости из Y-переменной (я вроде это забыл сказать), поэтому реабилитироваться буду примером:

In [14]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
 
np.random.seed(42069)
 
# Пусть у нас есть набор данных, где есть линейная зависимость y от X1 и X2
# При этом X1 и X2 тоже малясь зависимые
df = pd.DataFrame({'x1': np.random.uniform(0, 10, size=1000)})
df['x2'] = 4.9 + df['x1'] * 0.983 + 2.104 * np.random.normal(0, 1.35, size=1000)
df['y'] = 8.643 - 2.34 * df['x1'] + 3.35 * df['x2'] + np.random.normal(0, 1.65, size=1000)
df['const'] = 1
 
# Построим линейную регрессию-МНК из 1, X1 и X2
model = sm.OLS(
    endog=df['y'],
    exog=df[['const', 'x1', 'x2']]
).fit()

# Внимательно смотрим на коэффициент при X2
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                 1.754e+04
Date:                Wed, 09 Oct 2024   Prob (F-statistic):               0.00
Time:                        19:40:33   Log-Likelihood:                -1934.3
No. Observations:                1000   AIC:                             3875.
Df Residuals:                     997   BIC:                             3889.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.6842      0.139     62.606      0.000       8.412       8.956
x1            -2.3455      0.027    -88.274      0.000      -2.398      -2.293
x2             3.3544      0.019    178.202      0.000       3.317       3.391
==============================================================================
Omnibus:                        0.497   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.780   Jarque-Bera (JB):                0.394
Skew:                          -0.036   Prob(JB):                        0.821
Kurtosis:                       3.066   Cond. No.                         31.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
df

,x1,x2,y,const
0,0.305084,8.595233,37.454667,1
1,4.194086,8.942919,25.227052,1
2,8.776921,13.448463,32.000686,1
3,7.135028,7.653533,16.395354,1
4,3.239823,9.261669,30.338040,1
...,...,...,...,...
995,1.577805,8.699299,32.652689,1
996,8.186401,16.484709,43.966089,1
997,9.246921,11.438592,23.738685,1
998,4.609640,6.826222,20.713982,1


In [15]:
# Научим регрессию X1 на X2
model_x2 = sm.OLS(
    endog=df['x2'],
    exog=df[['const', 'x1']]
).fit()

# Научим регрессию X1 на y
model_yx1 = sm.OLS(
    endog=df['y'],
    exog=df[['const', 'x1']]
).fit()

In [16]:
# Полученными регрессиями "предскажем X2 и Y"

df['yx1'] = model_yx1.predict(df[['const', 'x1']])
df['x2x1'] = model_x2.predict(df[['const', 'x1']])

In [17]:
# А затем "отпилим" предсказание из данных
df['y_detrended'] = df['y'] - df['yx1']
df['x2_detrended'] = df['x2'] - df['x2x1']

In [18]:
# Учим модель на "очищенных" переменных и, о Боже, коэффициент при X2 остается "как был"
model_detrended = sm.OLS(
    endog=df['y_detrended'],
    exog=df[['const', 'x2_detrended']]
).fit()
model_detrended.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            y_detrended   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                 3.179e+04
Date:                Wed, 09 Oct 2024   Prob (F-statistic):               0.00
Time:                        19:41:38   Log-Likelihood:                -1934.3
No. Observations:                1000   AIC:                             3873.
Df Residuals:                     998   BIC:                             3882.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -2.158e-15      0.053  -4.07e-14      1.000      -0.104       0.104
x2_detrended     3.3544      0.019    178.292      0.000       3.318       3.391
==============================================================================
Omnibus:                        0.497   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.780   Jarque-Bera (JB):                0.394
Skew:                          -0.036   Prob(JB):                        0.821
Kurtosis:                       3.066   Cond. No.                         2.82
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# А теперь задачка :)

Возьмите данные c kaggle, например [отсюда](https://www.kaggle.com/code/malakalaabiad/house-prices-techniques/input) и удостоверьтесь, что FWL-теорема работает, но только не для случая одной переменной :)


# Про эквивалентность или не эквивалентность разных методов подсчета квантилей

Сгенерируйте 2 выборки длины ,например, 10000 из:

1. Нормального
2. Логнормального
3. Экспоненциального

Распределений с наперед заданными параметрами, так чтобы вы могли однозначно посчитать разницу медиан (используя теорвер и википедию)



Проверьте, какой по этим выборкам будет получаться 95% доверительный интервал на разницу медиан, если его посчитать с помощью:

1. Бутстрепа
2. [Подгонки](https://engineering.atspotify.com/2022/03/comparing-quantiles-at-scale-in-online-a-b-testing/) от Spotify
3. [Подгонки](https://www.evanmiller.org/bootstrapping-sample-medians.html) результатов бутстрепа от Эвана Миллера
4. [Метода Прайса-Боннетта](https://www.tandfonline.com/doi/abs/10.1080/00949650212140)

Что вы можете сказать о работоспособности методов?

(можно попробовать подать на вход какие-то другие распределения, как бы провести "стресс-тест" метода)
